In [29]:
import pandas as pd
from collections import defaultdict
import os
import openai
import requests
import time
from nameparser import HumanName
import re
from bs4 import BeautifulSoup
import re
from rapidfuzz import fuzz, process

In [30]:
base_path = r'C:\Projects\connecteddatahub\data'
presidents_path = os.path.join(base_path, 'cleaned_dataframes', 'presidents')

In [31]:
def normalize(x):
    return str(x).strip()

def clean_name(full_name):
    if not isinstance(full_name, str):
        return "Unknown"

    substrings_to_remove = [
        "Rev.", "SJ", "Sister", "Brother", "Father", "OP", "The Very",
        "Sr.", "O.P.", "Very Rev.", "Br.", "Dr.", "Md.", "S.J.", "Very",
        "M.D.", "O.P", "S.J", "J.R", "Jr.", "Jr ", "III", "Dr ", "The Honorable", 
        "Rev ", "Bishop ", "I.H.M", "O.S.U.", "F.S.C.", "O.S.A.*", "Sj", "Op ", 
        "Msgr", "O.S.U", "Ssj", "Ltgen", "Usaf", "(Ret)", "ret", "Sgt", "*", "(", ")", 
        "RSHM", "Usn", "Usa ", "Phd", "P.H.D", "Msgr.", "S.M", "S.M.", " R.S.C.J",
        "Rev.", "Sgt.", "O.S.A", "Most ", "The Hon", "Phd", "PhD", 'US', "USN", "Reverend",
        "CSC", "OSA", "OSO"
    ]

    # Remove unwanted substrings
    for substring in substrings_to_remove:
        full_name = full_name.replace(substring, "")

    # Remove single initials like "J." or "T "
    full_name = re.sub(r'\b[A-Z]\.?\s+', '', full_name)

    # Remove anything inside single or double quotes
    full_name = re.sub(r'["\'].*?["\']', '', full_name)

    # Collapse extra spaces
    full_name = re.sub(r'\s+', ' ', full_name)

    return full_name.strip()


In [32]:
presidents_dict = defaultdict(list)

for file in os.listdir(presidents_path):
    path = os.path.join(presidents_path, file)
    year = file.split("_")[0]
    df = pd.read_csv(path)

    df = df[df['PrimarySample'] == True]
    df["tempName"] = df["Name"].apply(lambda x: clean_name(normalize(x)))

    for _, row in df.iterrows():
        key = (row["AffiliationId"], row["Institution"])
        name = row["tempName"]
        presidents_dict[key].append((name, year))

presidents_dict = dict(presidents_dict)

for key, people in presidents_dict.items():
    print(key, people)

(71965598.0, 'Adelphi University') [('Matthew Goldstein', '1999'), ('Steven Isenberg', '2000'), ('Robert Allyn Scott', '2005'), ('Robert Allyn Scott', '2007'), ('Robert Allyn Scott', '2008'), ('Robert Allyn Scott', '2009'), ('Robert Allyn Scott', '2010'), ('Robert Allyn Scott', '2011'), ('Robert Allyn Scott', '2013'), ('Christine Marie Riordan', '2018')]
(181401687.0, 'American University') [('Benjamin Ladner', '1999'), ('Benjamin Ladner', '2000'), ('Beniamin Ladner', '2005'), ('Cornelius Kerwin', '2007'), ('Cornelius Kerwin', '2008'), ('Cornelius Kerwin', '2009'), ('Cornelius Kerwin', '2010'), ('Cornelius Kerwin', '2011'), ('Cornelius Kerwin', '2013'), ('Sylvia Mathews Burwell', '2018')]
(102298084.0, 'Andrews University') [('Niels-Erik Andreasen', '1999'), ('Niels-Erik Andreasen', '2000'), ('Niels-Erik Andreasen', '2005'), ('Niels-Erik Andreasen', '2007'), ('Niels-Erik Andreasen', '2008'), ('Niels-Erik Andreasen', '2009'), ('Niels-Erik Andreasen', '2010'), ('Niels-Erik Andreasen', '2

In [33]:
WIKI_API = "https://en.wikipedia.org/w/api.php"
HEADERS = {
    "User-Agent": "UniversityPresidentScraper/1.0 (contact: tykun1234@gmail.com)"
}

# ---------- Helpers ----------
def safe_request(params, retries=3, delay=1.0):
    for attempt in range(retries):
        try:
            resp = requests.get(WIKI_API, params=params, headers=HEADERS, timeout=10)
            if resp.status_code != 200:
                time.sleep(delay)
                continue
            return resp.json()
        except Exception as e:
            print("Request error:", e)
            time.sleep(delay)
    return None

def search_person(name, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": name,
        "format": "json",
        "srlimit": limit
    }
    data = safe_request(params)
    if not data or "query" not in data:
        return []
    return [hit["title"] for hit in data["query"]["search"]]

from bs4 import BeautifulSoup

def get_page_text(title):
    """Return the full plain-text article using action=parse."""
    params = {
        "action": "parse",
        "page": title,
        "prop": "text",
        "format": "json"
    }
    data = safe_request(params)
    if not data or "parse" not in data:
        return ""
    html_content = data["parse"]["text"]["*"]
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text(separator="\n")

def get_summary(title):
    url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + title.replace(" ", "_")
    resp = requests.get(url, headers=HEADERS, timeout=10)
    if resp.status_code == 200:
        return resp.json().get("extract", "")
    return ""

# ---------- Validation ----------
def validate_person_page(text, inst_name, pres_name, cand_title, threshold=80):
    t = text.lower()
    inst = inst_name.lower()

    # must mention role + institution
    if not any(kw in t and inst in t for kw in ["president", "chancellor", "principal", "rector"]):
        return False

    score = fuzz.token_sort_ratio(pres_name.lower(), cand_title.lower())
    if score >= threshold:
        return True

    intro_text = t[:1000]
    score_text = fuzz.token_sort_ratio(pres_name.lower(), intro_text)
    return score_text >= threshold

# ---------- Cache helper ----------
def get_from_cache(cache, pres_name, threshold=90):
    for cached_name, data in cache.items():
        if fuzz.token_sort_ratio(pres_name.lower(), cached_name.lower()) >= threshold:
            return data, cached_name
    return None, None

# ---------- Main ----------
def build_president_page_map(presidents_dict):
    result = {}
    cache = {}

    for (inst_id, inst_name), pres_list in presidents_dict.items():
        print("\nProcessing", inst_name, "...")
        result[inst_name] = {}
        for pres, year in pres_list:
            # cache lookup
            cached_data, cached_key = get_from_cache(cache, pres, threshold=90)
            if cached_data:
                result[inst_name][pres] = cached_data
                print(" ", pres, "->", cached_data["page"], "(cached as", cached_key, ")")
                continue

            found_title = None
            search_names = [pres]

            # if 3 parts, add first+last variant
            parts = pres.split()
            if len(parts) == 3:
                search_names.append(parts[0] + " " + parts[2])

            # try all candidate names
            for search_name in search_names:
                candidates = search_person(search_name, limit=10)
                for cand in candidates:
                    text = get_page_text(cand)
                    if text and validate_person_page(text, inst_name, pres, cand):
                        found_title = cand
                        break
                if found_title:
                    break  # stop if we found a match

            if found_title:
                found_summary = get_summary(found_title)
                found_text = get_page_text(found_title)  # full article text
            else:
                found_summary, found_text = None, None

            cache[pres] = {
                "page": found_title,
                "summary": found_summary,
                "text": found_text  # entire page text
            }
            result[inst_name][pres] = cache[pres]
            print(" ", pres, "->", found_title)
            time.sleep(0.5)

    return result


In [34]:
subset_size = len(presidents_dict)
subset_dict = dict(list(presidents_dict.items())[:subset_size])

president_pages = build_president_page_map(subset_dict)
# president_pages = build_president_page_map(presidents_dict)


Processing Adelphi University ...
  Matthew Goldstein -> Matthew Goldstein
  Steven Isenberg -> None
  Robert Allyn Scott -> Robert A. Scott
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Robert Allyn Scott -> Robert A. Scott (cached as Robert Allyn Scott )
  Christine Marie Riordan -> Christine Riordan

Processing American University ...
  Benjamin Ladner -> Benjamin Ladner
  Benjamin Ladner -> Benjamin Ladner (cached as Benjamin Ladner )
  Beniamin Ladner -> Benjamin Ladner (cached as Benjamin Ladner )
  Cornelius Kerwin -> Cornelius M. Kerwin
  Cornelius Kerwin -> Cornelius M. Kerwin (cached as Cornelius Kerwin )
  Cornelius Kerwin -> Cornelius M. Kerwin (cached as Corneli

ReadTimeout: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=10)

In [ ]:
print(president_pages["Adelphi University"]["Robert Allyn Scott"])



{'page': 'Robert A. Scott', 'summary': 'Robert Allyn Scott is an American educator and academic administrator who served as ninth president of Adelphi University located in Garden City, New York from 2000–2015.', 'text': 'American educator and academic administrator\n\n\nFor other people named Robert Scott, see \nRobert Scott (disambiguation)\n.\n\n\n\n\n\n\nRobert A. Scott\n9th President of \nAdelphi University\nIn office\nJuly 1, 2000\xa0– July 1, 2015\nPreceded by\nIgor Webb \n Steven Isenberg (interim)\nSucceeded by\nChristine Riordan\n2nd President of \nRamapo College\nIn office\nJune 1, 1985\xa0– June 30, 2000\nPreceded by\nGeorge T. Potter\nSucceeded by\nRodney David Smith\n\n\nPersonal details\nEducation\nBucknell University\n (\nBA\n)\nCornell University\n (\nPhD\n)\n\nScientific career\nFields\nHigher Education\n\n\n\n\n\n\nRobert Allyn Scott\n is an American educator and academic administrator who served as ninth president of \nAdelphi University\n located in \nGarden City, 

In [ ]:
import json

output_dir = "C:\Projects\connecteddatahub\data\scraped"
output_path = os.path.join(output_dir, "president_pages.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(president_pages, f, ensure_ascii=False, indent=2)

print("Saved to", output_path)


Saved to C:\Projects\connecteddatahub\data\scraped\president_pages.json


<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\tykun\AppData\Local\Temp\ipykernel_3588\3277999740.py:3: SyntaxWarning: invalid escape sequence '\P'
  output_dir = "C:\Projects\connecteddatahub\data\scraped"


In [ ]:
# import os
# import json
# import google.generativeai as genai

# genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# model = genai.GenerativeModel("models/gemini-2.5-flash-lite")

# def extract_person_info(inst_id, pres_name, year, data):
#     """Send biography text to Gemini and extract structured info."""
#     text = data.get("text") or data.get("summary") or ""
#     if not text or len(text.strip()) < 50:
#         print(f"Skipping API call for {pres_name} due to insufficient text.")
#         return None

#     prompt = f"""
#         You are an information extraction system.  
#         Given the biography text below, extract structured information for the following fields.  
#         If information is not available, use an empty list [] or null where specified.  

#         Important rules:
#         - Output must be strictly valid JSON.  
#         - Use the same keys and structure every time.  
#         - Do not write sentences. Only use the controlled vocabularies below.  
#         - If multiple values apply, return them as a list.  
#         - Do not invent information not present in the text.  

#         ### Required Keys
#         - inst_id: string (passed in, do not change)  
#         - pres_name: string (passed in, do not change)  
#         - year: string (passed in, do not change)  
#         - degree: list of objects with fields:
#         - level: one of [ "bachelors", "masters", "phd", "jd", "md", "other" ]
#         - discipline: one of [ "theology_religion", "philosophy", "history", "literature_english", "linguistics", "education", "political_science", "sociology", "psychology", "economics", "business_management", "law", "medicine", "biology", "chemistry", "physics", "mathematics", "engineering", "computer_science", "environmental_science", "public_health", "arts_music", "other" ]
#         - alumni_status: true, false, or null  
#         - scientific_career: list of disciplines (must be chosen from the same discipline list above)  

#         ### Biography Text
#         \"\"\" 
#         {text} 
#         \"\"\"  

#         ### Input Metadata
#         Institution ID: {inst_id}  
#         President Name: {pres_name}  
#         Year: {year}  

#         ### Example Output
#         ```json
#         {{
#         "inst_id": "423412421",
#         "pres_name": "Robert Sloan",
#         "year": "2005",
# \        "degree": [
#             {{"level": "bachelors", "discipline": "theology_religion"}},
#             {{"level": "masters", "discipline": "theology_religion"}},
#             {{"level": "phd", "discipline": "theology_religion"}}
#         ],
#         "alumni_status": true,
#         "scientific_career": ["theology_religion", "education"]
#         }}
#         ###"""
#     response = model.generate_content(
#     prompt,
#         generation_config=genai.types.GenerationConfig(
#             temperature=0,
#             response_mime_type="application/json"
#         )
#     )

#     content = response.text.strip()
#     parsed = json.loads(content)

#     print(f"Extraction for {pres_name}:")
#     print(json.dumps(parsed, indent=2))

#     return parsed


<>:62: SyntaxWarning: invalid escape sequence '\ '
<>:62: SyntaxWarning: invalid escape sequence '\ '
C:\Users\tykun\AppData\Local\Temp\ipykernel_3588\191424491.py:62: SyntaxWarning: invalid escape sequence '\ '
  ###"""


In [ ]:
# import google.generativeai as genai
# import os

# # Make sure your GEMINI_API_KEY is set as an environment variable
# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# print("Available models:")
# for m in genai.list_models():
#   # Check if the model supports the 'generateContent' method
#   if 'generateContent' in m.supported_generation_methods:
#     print(f"- {m.name}")

In [ ]:
# results = []
# cache = {}  # cache of already-processed presidents

# for (inst_id, inst_name), pres_list in presidents_dict.items():
#     for pres_name, year in pres_list:
#         if inst_name in president_pages and pres_name in president_pages[inst_name]:
#             # Use (pres_name) as the cache key — could also do (inst_id, pres_name) if needed
#             cache_key = pres_name  

#             if cache_key in cache:
#                 print(f"Reusing cached data for {pres_name}")
#                 extracted = cache[cache_key]
#             else:
#                 pdata = president_pages[inst_name][pres_name]
#                 extracted = extract_person_info(inst_id, pres_name, year, pdata)
#                 if extracted:
#                     cache[cache_key] = extracted  # save to cache

#             if extracted:
#                 results.append(extracted)


Extraction for Matthew Goldstein:
{
  "inst_id": "71965598.0",
  "pres_name": "Matthew Goldstein",
  "year": "1999",
  "degree": [
    {
      "level": "bachelors",
      "discipline": "mathematics"
    },
    {
      "level": "phd",
      "discipline": "mathematics"
    }
  ],
  "alumni_status": true,
  "scientific_career": [
    "mathematics",
    "statistics"
  ]
}
Skipping API call for Steven Isenberg due to insufficient text.
Extraction for Robert Allyn Scott:
{
  "inst_id": "71965598.0",
  "pres_name": "Robert Allyn Scott",
  "year": "2005",
  "degree": [
    {
      "level": "bachelors",
      "discipline": "literature_english"
    },
    {
      "level": "phd",
      "discipline": "sociology"
    },
    {
      "level": "other",
      "discipline": "other"
    }
  ],
  "alumni_status": null,
  "scientific_career": [
    "education"
  ]
}
Reusing cached data for Robert Allyn Scott
Reusing cached data for Robert Allyn Scott
Reusing cached data for Robert Allyn Scott
Reusing cache

C:\Users\tykun\AppData\Local\Temp\ipykernel_3588\191424491.py:62: SyntaxWarning: invalid escape sequence '\ '
  ###"""


KeyboardInterrupt: 